In [ ]:
# 1. Install required packages (The -q flag means quiet installation)
! pip install ultralytics -q
! pip install streamlit -q  # <<<--- ADDED: Explicitly install Streamlit
! pip install pyngrok -q
! pip install roboflow -q
! pip install opencv-python-headless -q # Ensure CV2 is ready for Streamlit

# 2. Mount Google Drive for saving model weights
from google.colab import drive
drive.mount('/content/drive')

# 3. Clean up any previous dataset folder to ensure a clean download
!rm -rf mask-hairnet-gloves-iopkp-1  

# 4. Load the dataset
from roboflow import Roboflow
rf = Roboflow(api_key="vLBWuwn1rsPb9kU40N44")
project = rf.workspace("workshopr").project("mask-hairnet-gloves-iopkp")
version = project.version(1)

# CRITICAL: Download in standard 'yolov11' format
dataset = version.download("yolov11") 

# Define the paths for the next steps
DATASET_DIR = dataset.location
DATA_YAML_PATH = f"{DATASET_DIR}/data.yaml"

print(f"\n✅ Dataset successfully downloaded to: {DATASET_DIR}")
print(f"data.yaml path: {DATA_YAML_PATH}")

In [ ]:
from ultralytics import YOLO
import torch

# Ensure Colab is using a GPU (Runtime -> Change runtime type)
print(f"CUDA Available: {torch.cuda.is_available()}")

# Load a pre-trained detection model (nano-size)
model = YOLO('yolo11n.pt')

# --- Start Training ---
# - task='detect': Activates the standard Axis-Aligned Bounding Box mode
results = model.train(
    task='detect',
    data=DATA_YAML_PATH,
    epochs=100, 
    imgsz=640,
    patience=20, 
    batch=16, 
    name='ppe_yolo11_detect_v1'
)

# --- Backup Model ---
MODEL_SAVE_PATH = '/content/runs/detect/ppe_yolo11_detect_v1/weights/best.pt'
! cp {MODEL_SAVE_PATH} /content/drive/MyDrive/best_ppe_yolo11_model.pt
print(f"\nModel saved to Drive: /content/drive/MyDrive/best_ppe_yolo11_model.pt")

In [ ]:
from ultralytics import YOLO
import glob
from IPython.display import Image, display
from pathlib import Path

# Path to the final trained model weights
BEST_MODEL_PATH = '/content/runs/detect/ppe_yolo11_detect_v1/weights/best.pt'
model = YOLO(BEST_MODEL_PATH, task='detect')

# 1. Validate the model (Calculates required mAP metrics)
print("\n--- Running Final Validation ---")
metrics = model.val(
    data=DATA_YAML_PATH,
    split='test' # Validate against the test set
)
print(f"\n✅ Model mAP50-95 Score: {metrics.box.map}")
print(f"✅ Model mAP50 Score: {metrics.box.map50}")


# 2. Run Inference on a sample image (optional visualization)
SAMPLE_IMAGE_DIR = f'{DATASET_DIR}/test/images'
sample_images = glob.glob(f'{SAMPLE_IMAGE_DIR}/*.jpg')

if sample_images:
    sample_path = sample_images[0]
    print(f"\n--- Running Inference on Sample Image: {Path(sample_path).name} ---")
    
    # Run prediction and save results
    results = model.predict(sample_path, save=True, conf=0.25, name='sample_inference', exist_ok=True)
    
    # Display the annotated image
    display_path = Path('/content/runs/detect/sample_inference') / Path(sample_path).name
    print(f"\n✅ Displaying annotated image:")
    display(Image(filename=str(display_path)))
else:
    print("\n⚠️ Could not find a sample image in the test set for visualization.")

In [ ]:
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import cv2
import tempfile
import numpy as np
import yaml
from pathlib import Path

# --- Configuration (CRITICAL: Must match your training setup) ---
MODEL_PATH = 'best_ppe_yolo11_model.pt' 
# Default path created by Roboflow in Step 1
DATA_YAML_PATH = '/content/mask-hairnet-gloves-iopkp-1/data.yaml' 

# Load class names from the Roboflow data.yaml
try:
    with open(DATA_YAML_PATH, 'r') as f:
        data_config = yaml.safe_load(f)
    CLASS_NAMES = data_config.get('names', [])
    if not CLASS_NAMES:
        st.error("Could not load class names from data.yaml.")
        CLASS_NAMES = ['mask', 'glove', 'haircap'] # Fallback
except Exception as e:
    st.error(f"Error loading data.yaml: {e}. Using default class names.")
    CLASS_NAMES = ['mask', 'glove', 'haircap'] # Fallback

# Load the model once at startup (Streamlit caching for performance)
@st.cache_resource
def load_model():
    # Load the model using the correct task='detect'
    return YOLO(MODEL_PATH, task='detect') 

MODEL = load_model()

# Set Streamlit page layout
st.set_page_config(
    page_title="YOLOv11 PPE Detector",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- Processing Logic: Runs detection and draws boxes ---
def process_frame(frame, conf_threshold):
    """Runs standard detection inference and draws bounding boxes on the frame."""
    
    # Run prediction, forcing GPU if available (device='0')
    results = MODEL.predict(frame, conf=conf_threshold, verbose=False, device='0')[0]
    
    annotator = Annotator(frame, line_width=2, example=CLASS_NAMES)
    
    # Object Counter initialization (Required feature)
    object_count = {name: 0 for name in CLASS_NAMES}
    
    if results.boxes:
        for box in results.boxes:
            c = int(box.cls[0]) # Class index
            label = f"{CLASS_NAMES[c]} {box.conf[0]:.2f}"
            
            # Draw the Axis-Aligned Bounding Box (AABB)
            xyxy = box.xyxy[0].tolist()
            annotator.box_label(xyxy, label, color=colors(c, True))
            
            # Update object count
            if 0 <= c < len(CLASS_NAMES):
                object_count[CLASS_NAMES[c]] += 1
            
    # Display the object count in the sidebar (Required feature)
    st.sidebar.subheader("Detected Objects Count")
    for name, count in object_count.items():
        st.sidebar.markdown(f"**{name.capitalize()}:** `{count}`")

    return annotator.result()


# --- Main Dashboard Structure ---
st.title("🛡️ Safety Gear (PPE) Object Detector")
st.sidebar.header("Model Configuration")

# Sidebar controls (Required feature: Confidence slider)
confidence = st.sidebar.slider(
    "Select Confidence Threshold (Minimum prediction score)", 
    min_value=0.01, 
    max_value=1.0, 
    value=0.25 # Default value
)

st.sidebar.markdown("---")
source_option = st.sidebar.radio("Select Input Source", ('Image Upload', 'Video Upload'))

# --- Source Handlers ---
if source_option == 'Image Upload':
    uploaded_file = st.file_uploader("Upload an image for detection", type=['jpg', 'jpeg', 'png'])
    if uploaded_file is not None:
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        image = cv2.imdecode(file_bytes, 1)

        st.subheader("Detected Image")
        with st.spinner('Processing Image...'):
            processed_image = process_frame(image, confidence)
            st.image(processed_image, channels="BGR", use_column_width=True)
            st.success("Detection complete.")

elif source_option == 'Video Upload':
    # This is crucial for your demo, as it shows real-time processing
    uploaded_file = st.file_uploader("Upload a video for detection (e.g., MP4)", type=['mp4', 'avi', 'mov'])
    if uploaded_file is not None:
        st.subheader("Video Detection Feed (Real-time Simulation)")
        
        # Save uploaded video to a temporary file
        tfile = tempfile.NamedTemporaryFile(delete=False)
        tfile.write(uploaded_file.read())
        
        cap = cv2.VideoCapture(tfile.name)
        st_frame = st.empty()

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Process frame and display
            processed_frame = process_frame(frame, confidence)
            st_frame.image(processed_frame, channels="BGR", use_column_width=True)
            
        cap.release()
        st.success("Video processing complete.")

In [ ]:
# 1. Copy the trained model weights to the current directory (needed by app.py)
! cp /content/runs/detect/ppe_yolo11_detect_v1/weights/best.pt /content/best_ppe_yolo11_model.pt

# The Streamlit server runs on port 8501
from pyngrok import ngrok
import subprocess
import time

# 2. Terminate any previous ngrok tunnels
try:
    ngrok.kill()
    print("Previous ngrok tunnels terminated.")
except Exception as e:
    print(f"Could not terminate ngrok: {e}")

# 3. Start the Streamlit app in the background
print("Starting Streamlit app...")
# Use 'python3 -m streamlit' to ensure the correct module is executed
proc = subprocess.Popen(
    ["python3", "-m", "streamlit", "run", "app.py", "--logger.level", "error"],
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE, 
    text=True,
    shell=False
)
time.sleep(8) # Wait longer for Streamlit to fully initialize

# 4. Check if the Streamlit process started correctly
if proc.poll() is not None:
    # If the process stopped immediately, print the captured logs
    print("🚨 FATAL ERROR: Streamlit failed to start. (This should not happen now)")
    stdout, stderr = proc.communicate()
    print("\n--- Streamlit STDOUT (Logs) ---")
    print(stdout)
    print("\n--- Streamlit STDERR (Error Traceback) ---")
    print(stderr)
else:
    # 5. Start the ngrok tunnel on the Streamlit port
    print("Attempting to connect via ngrok...")
    try:
        # NOTE: ngrok may ask you to set an auth token if you haven't done so.
        public_url = ngrok.connect(8501)
        print(f"🎉 Streamlit Dashboard Live at: {public_url}")
        print("💡 NOTE: Click the link above. Keep this cell running to keep the link active for your demo.")
        # Keep the cell alive indefinitely
        time.sleep(9999999) 
    except Exception as e:
        print(f"🚨 NGROK ERROR: {e}")
        print("Check if you need to set your ngrok auth token. Use !ngrok config add-authtoken <YOUR_TOKEN> in a new cell.")